## Get the incriminated informations 

In [ ]:
from component import widget as cw
from component import parameter as cp
from pathlib import Path

folder_w = cw.FolderSelect(folder=cp.result_dir / "bfast_gpu_debug")
folder_w

## check the content 

if the foder result of this cell is false you shoud not continue 

In [ ]:
folder = Path(folder_w.v_model)

monitoring_params_f = folder / "monitor_params.json"
dates_f = folder / "dates.json"
data_f = folder / "data.npy"

[f.is_file() for f in [monitoring_params_f, dates_f, data_f]]

## prepare the data

read everything and cast back json datetime strings as datetime objects 

In [ ]:
import json
from datetime import datetime as dt
import numpy as np

In [ ]:
# monitoring_param

monitoring_params = json.loads(monitoring_params_f.read_text())
monitoring_params.update(
    start_monitor=dt.strptime(monitoring_params["start_monitor"], "%Y-%m-%d")
)
monitoring_params

In [ ]:
# dates
dates = [dt.strptime(d, "%Y-%m-%d") for d in json.loads(dates_f.read_text())]
# dates

In [ ]:
# data
data = np.load(data_f)
print(f"shape: {data.shape}")

nb_image = data.shape[0]
print(f"nb images: {data.shape[0]}")

## run bfast model and display magnitude results 

In [ ]:
from bfast import BFASTMonitor

model = BFASTMonitor(**monitoring_params)
model.fit(data, dates)

results = model.magnitudes

results.shape

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(results, cmap="viridis", interpolation="nearest")
plt.colorbar()
plt.show()

In [ ]:
## check the images

from matplotlib import pyplot as plt

min_ = 0  # change the value to see less or more images
max_ = nb_image

for i in range(min_, min(max_, nb_image)):

    plt.imshow(data[i], cmap="viridis", interpolation="nearest")
    plt.title(dates[i].strftime("%Y-%m-%d"))
    plt.colorbar()
    plt.show()

## run all the tile separately

I want to know if the bug is coming from the writing function 

In [ ]:
from component import widget as cw
from component import parameter as cp
from pathlib import Path

folder_w = cw.FolderSelect(folder=cp.result_dir / "bfast_gpu_debug")
folder_w

In [ ]:
folder = Path(folder_w.v_model)

for el in folder.glob("*/"):

    int(el.stem)

print("ok")

In [ ]:
import json
from datetime import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
from bfast import BFASTMonitor
from tqdm import tqdm


for el in tqdm(folder.glob("*/")):

    output = el / "magnitudes.npy"

    if output.is_file():
        out = np.load(output)
    else:

        # get the files to read
        monitoring_params_f = el / "monitor_params.json"
        dates_f = el / "dates.json"
        data_f = el / "data.npy"

        monitoring_params = json.loads(monitoring_params_f.read_text())
        monitoring_params.update(
            start_monitor=dt.strptime(monitoring_params["start_monitor"], "%Y-%m-%d")
        )

        dates = [dt.strptime(d, "%Y-%m-%d") for d in json.loads(dates_f.read_text())]

        data = np.load(data_f)

        model = BFASTMonitor(**monitoring_params)
        model.fit(data, dates)

        out = model.magnitudes

        np.save(output, out)

    # display the result on a plot
    plt.imshow(out, cmap="viridis", interpolation="nearest", vmin=-10, vmax=10)
    plt.title(f"window {el.stem}")
    plt.colorbar()
    plt.show()